In [74]:
import os
import shutil
import nbconvert
import nbformat

# set the current directory to the where the code snippets are
dest_root = os.path.join(os.getcwd(), 'docs')
notes_rel = '../../Notes/Code Snippets/'
source_root = os.path.join(os.getcwd(), notes_rel)
os.chdir(source_root)

In [44]:
# Three resources on how to do this:
# 1st: https://github.com/jupyter/nbconvert/issues/699#issuecomment-372441219       this works
# 2nd: https://gist.github.com/connerxyz/df8e1a2d3915aade869c968725c15cf3           this also works
# 3rd: https://nbconvert.readthedocs.io/en/latest/nbconvert_library.html#Example    couldn't get this to work

In [56]:
def ipynb_to_markdown(src_file, dest):

    file_name = src_file[src_file.rfind('/')+1:]

    with open(src_file) as nb_file:
        nb_contents = nb_file.read()

    # Convert using the ordinary exporter
    notebook = nbformat.reads(nb_contents, as_version=4)
    exporter = nbconvert.MarkdownExporter()
    body, res = exporter.from_notebook_node(notebook)

    # Fix up the markdown (included images to subfolder)
    for key in res['outputs']:
        body = body.replace(f'({key})', f'({file_name}-files/{key})')

    #  Write markdown file to disk
    with open(os.path.join(dest,file_name+'.md'), 'w') as output_file:
        output_file.write(body)

    #  Write images to disk
    for key in res['outputs']:
        os.makedirs(os.path.join(dest,file_name+'-files'), exist_ok=True) # create folder
        fname = os.path.join(*[dest,file_name+'-files',key])
        with open(fname, 'wb') as output_file:
            output_file.write(res['outputs'][key])

In [76]:
# start with the root directory, save the relative path in a variable

# - get a list of directory contents
# - go through the list of directory contents
#     - if entry is a file, process the file:
#         determine the target folder (including file name, and parameters to be added into the file)
#         if the file is a jupyter file:
#             convert to markdown -> and copy into the folder
#         if the file is a markdown file:
#             copy into the folder
#         open the file and edit the paramters
#     - if entry is a folder, then process the folder:
        

def process_folder(cur_loc):
    p = os.listdir(cur_loc) # get list of all file and folders in the directory
    if '.obsidian' in p:    # housekeeping: exclude certain folders
        p.remove('.obsidian') # clean list

    # go through the list:
    for i in p:
        if os.path.isdir(cur_loc+'/'+i): # i is a directory
            #print(cur_loc+'/'+i)
            process_folder(cur_loc+'/'+i)
        else: # i is a file
            process_file(cur_loc+'/'+i)



def process_file(filepname):    # filepname is always relative to the source_root
        file_extension = filepname[filepname.rfind('.')+1:]
        file_name = filepname[filepname.rfind('/')+1:]
        file_loc = filepname[2:filepname.rfind('/')+1]
        #print(file_extension, file_name, file_path)
        if file_extension == 'md':
            os.makedirs(os.path.join(dest_root,file_loc), exist_ok=True)                            # create/ensure destination directory exists
            shutil.copy2(os.path.join(*[source_root,file_loc,file_name]), os.path.join(dest_root,file_loc))     # copy the file there
            write_header(filepname)
            process_psr_markup(filepname)
            
        if file_extension == 'ipynb':
            os.makedirs(os.path.join(dest_root,file_loc), exist_ok=True)                            # create/ensure destination directory exists
            ipynb_to_markdown(os.path.join(*[source_root,file_loc,file_name]),os.path.join(dest_root,file_loc)) # convert the file
            write_header(filepname+'.md')
            process_psr_markup(filepname+'.md')



def write_header(filepname):
    # file_extension = filepname[filepname.rfind('.')+1:]
    # file_name = filepname[filepname.rfind('/')+1:]
    file_loc = filepname[2:filepname.rfind('/')+1]
    file_name_wo_ext = filepname[filepname.rfind('/')+1:filepname.rfind('.')]


    # CREATE THE HEADER
    # has_children: 'true' wenn es einen Ordner mit dem gleichen Namen wie das Dokument gibt; ansonsten 'false'
    # parent: Name des Ordners in dem das Dokument liegt, ausser der Ordner ist 'docs' (oder leer), dann wird die Zeile 'parent' ausgelassen
    # nav_order: Zahl vor dem Titel der Datei (= vor dem ersten Leerzeichen)
    # title: Name der Datei (ohne nav_order; = nach dem ersten Leerzeichen)

    header = ['---\n'] # start the header

    # has children
    haschildren = os.path.isdir(os.path.join(file_loc,file_name_wo_ext)) # if the folder existed, then it would have the same name as the file
    if haschildren:
        # print(f'The file {filepname} has a child')
        header.extend(['has_children: true \n'])
    
    # parent:
    parent_name = file_loc[file_loc[:-1].rfind('/')+1:-1]
    if parent_name != 'docs' and parent_name != '' and parent_name != ' ': # the name of the root folder:
        # print(parent_name)
        header.extend([f'parent: {parent_name} \n'])

    # title:
    title = file_name_wo_ext[file_name_wo_ext.find(' ')+1:]
    header.extend([f'title: {title} \n'])
    
    # order:
    navorder = file_name_wo_ext[:file_name_wo_ext.find(' ')]
    header.extend([f'nav_order: {navorder} \n'])
    
    header.extend(['---\n']) # finish the header
    header.extend(['\n'])


    # WRITE THE FILE
    with open(os.path.join(dest_root,filepname), 'r') as f:
        text = f.readlines()
    with open(os.path.join(dest_root,filepname), 'w') as f:    
        f.writelines(header+text)


def process_psr_markup(filepname):
    file_loc = filepname[2:filepname.rfind('/')+1]
    file_name_wo_ext = filepname[filepname.rfind('/')+1:filepname.rfind('.')]

    # LOAD THE FILE
    with open(os.path.join(dest_root,filepname), 'r') as f:
        text = f.readlines()
    
    # CREATE THE TOC:
    # check if a toc should be inserted and - if so - insert it into the text
    #  a toc should be inserted if in the document there is a line '{:psr: toc here}'
    #  in this case, replace the line '{:psr: toc here}' with the text in toc_insert
    line_nr = next((i for i, x in enumerate(text) if x[:16]=="{:psr: toc here}"), -1) # search for the line
    if line_nr != -1: # check if a toc should be inserted (= in case a line was found)
        toc_insert = ['{: .no_toc } \n', '<details open markdown="block"> \n', '  <summary> \n', '    Table of contents \n', '  </summary> \n', '  {: .text-delta } \n', '1. TOC \n', '{:toc} \n', '</details>] \n']    
        text = text[:line_nr]+toc_insert+text[line_nr+1:] # take out the line and insert the toc
        
    # SAVE THE FILE
    with open(os.path.join(dest_root,filepname), 'w') as f:    
        f.writelines(text)



In [77]:
os.chdir(source_root)
process_folder('.')

NameError: name 'header' is not defined

In [63]:
source_root

'c:\\Users\\Niklas\\OneDrive\\Notes\\Code Snippets\\../../Notes/Code Snippets/'

In [ ]:
    toc_insert = [
        '{: .no_toc } \n',
        '<details open markdown="block"> \n',
        '  <summary> \n',
        '    Table of contents \n',
        '  </summary> \n',
        '  {: .text-delta } \n',
        '1. TOC \n',
        '{:toc} \n',
        '</details>] \n']
    print(toc_insert)

    # toc:
    #   check if a toc should be inserted and - if so - insert it into the text
    if len(navorder)>3:
        if navorder[-3:] == '_TOC':
            line_nr = next((i for i, x in enumerate(text) if x[:2]=="# "), [-1]) # find the line of the first heading
            text = text[:line_nr+1]
            line_nr = next((line_nr for line_nr, line in enumerate(text) if [line[:2]=="# "]), -1) # find the line of the first heading
            text = text[:line_nr+1]+toc_insert+text[:line_nr+1]

In [11]:
text = [
    '{: .no_toc } \n',
    '<details open markdown="block"> \n',
    '  <summary> \n',
    '    Table of contents \n',
    '  </summary> \n',
    '  {: .text-delta } \n',
    '1. TOC \n',
    '{:toc} \n',
    '</details>] \n']

In [14]:
text[1]

'<details open markdown="block"> \n'

In [20]:
next((x for x in text if x[:2]=="  "), [-1])

'  <summary> \n'

In [40]:
line_nr = next((i for i, x in enumerate(text) if x[:2]=="as"), -1) # find the line of the first heading

In [41]:
if line_nr <-1:
    print('ja')

In [42]:
text[:line_nr+1]+['asdf \n','dfdf \n']+text[line_nr+1:]

['asdf \n',
 'dfdf \n',
 '{: .no_toc } \n',
 '<details open markdown="block"> \n',
 '  <summary> \n',
 '    Table of contents \n',
 '  </summary> \n',
 '  {: .text-delta } \n',
 '1. TOC \n',
 '{:toc} \n',
 '</details>] \n']

In [ ]:
import nbconvert
import nbformat

with open('health-insurance.ipynb') as nb_file:
    nb_contents = nb_file.read()

# Convert using the ordinary exporter
notebook = nbformat.reads(nb_contents, as_version=4)
exporter = nbconvert.MarkdownExporter()
body, res = exporter.from_notebook_node(notebook)

# Fix up the markdown (included images to subfolder)
for key in res['outputs']:
    body = body.replace(f'({key})', f'(/logs/{key})')

#  Write markdown file to disk
with open('health-insurance.ipynb.md', 'w') as output_file:
    output_file.write(body)

#  Write images to disk
for key in res['outputs']:
    fname = 'asdf'+key
    with open(fname, 'wb') as output_file:
        output_file.write(res['outputs'][key])

In [39]:
filepname="Pandas.md"
os.getcwd()

with open(filepname, 'r') as f:
    text = f.readlines()
    
    # new_text = "\n".join([line if line != orig else new for line in text])

new_text = ['---\n']
new_text.extend(['has_children: asdf \n'])
new_text

['---\n', 'has_children: asdf']

In [10]:
# root directory
p = os.listdir() # returns list
p.remove('.obsidian')

# go through the root list:
for i in p:
    if os.path.isdir(i): # i is a directory
        print(f'directory name: {i}')
        
        # in the directory get a list of all files
        inside = os.listdir(os.path.join(notes_abs, i)) # returns list
        for count, value in enumerate(inside):
            
            value is the file name
                decompose the file name into parts: section and title
            
            open the file and read the first line. 
                if the first line starts with {jpy-insert}, then this document is just a shell
                run nb-convert and save the resulting file(s) in the jekyll folder
            
            open the file and add the header: insert the header parts (based on value, section, and title)

    else: # i is a file
        print(f'file name: {i}')

directory name: Datatypes
['Dictionaries.md', 'Lists  - apply a function to all items in a list.md', 'Lists  - create an empty list of a certain size.md', 'Lists  - insert and take from lists.md', 'Lists - compare lists.md', 'Lists - enumerate.md', 'Lists - find in list.md', 'Lists - parsing lists from text files.md', 'Lists - sort lists.md', 'Named Tuples.md', 'Strings  - count occurences in a string.md', 'Strings - split into different parts.md']
directory name: Machine Learning
['index.md']
directory name: Pandas
['Pandas - 0 Load data.md', 'Pandas - 0b Deal with dates and geodata.md', 'Pandas - 0c Use feather.md', 'Pandas - 1 Describe and explore dataset.md', 'Pandas - 1b Deal with missing observations.md', 'Pandas - 1b2 Deal with duplicates.md', 'Pandas - 1b3 Deal with categorical data.md', 'Pandas - 1b4 Deal with missing rows.md', 'Pandas - 1c Select observations.md', 'Pandas - 1d1 Wrangle data 1 - Single rows, columns, or indices.md', 'Pandas - 1d2 Wrangle data 2 - Groupby and m